# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     María Fernanda Muñoz Sevilla|   |
| **Fecha**     10/09/25 |   |
| **Expediente*751190*   |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [66]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [68]:
df= pd.read_csv("C:/Users/munoz/Downloads/lab_apre_est/Advertising.csv")
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [70]:
df=df.drop(columns='Unnamed: 0')
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [72]:
tv=df.TV.values.reshape([-1,1])
radio=df.radio.values.reshape([-1,1])
newsp=df.newspaper.values.reshape([-1,1])
salesy=df.sales


## Sin penalización

In [105]:
n1=len(salesy)
unos=np.ones([n1,1])
X=np.hstack([unos,tv,radio,newsp])


In [143]:
lr = LinearRegression()
lr.fit(X, salesy)
beta0 = lr.intercept_
betas = lr.coef_
print("Intercepto Beta_0:", beta0)
print("Coeficientes:", betas)

Intercepto Beta_0: 2.9388893694594085
Coeficientes: [ 0.          0.04576465  0.18853002 -0.00103749]


In [145]:
rss=np.sum((salesy-lr.predict(X))**2)
print("RSS: ", rss)
n=len(salesy)
var=rss/(n-2)
print(var)

RSS:  556.8252629021872
2.812248802536299


In [147]:
xmed = np.mean(X)
seb0=((var)*((1/n1)+(((xmed)**2)/(np.sum((X-xmed)**2)))))**0.5
print("Error estandar de beta 0: ",seb0)

Error estandar de beta 0:  0.12558553995430408


In [149]:
seb1=(var/(np.sum((X-xmed)**2)))**0.5
print("Error estandar de beta 1: ",seb1)

Error estandar de beta 1:  0.0008195361845936359


In [151]:
t1=betas/seb1
print("Valor t de beta 1: ",t1)

Valor t de beta 1:  [  0.          55.84212914 230.04477467  -1.26595148]


In [194]:
from scipy import stats
p_sp=2*(1-stats.t.cdf(np.abs(t1),n1-2))
print("p-value: ", p_sp)

p-value:  [1.         0.         0.         0.20701794]


In [155]:
y_pred = lr.predict(X)
r2=r2_score(salesy,y_pred)
print("R^2: ", r2)

R^2:  0.8972106381789522


In [157]:
ols= sm.OLS(salesy,X)
results=ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        11:23:29   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Con penalización L2

In [160]:
ridge = Ridge(alpha=0.5)
ridge.fit(X, salesy)
inter=ridge.intercept_
print("Intercepto: ",inter) 

Intercepto:  2.93892841431542


In [162]:
coef=ridge.coef_
print("Coeficientes: ", coef)

Coeficientes:  [ 0.          0.04576464  0.18852756 -0.00103689]


In [164]:
rss2=np.sum((salesy-ridge.predict(X))**2)
print("RSS: ",rss2)
n=len(salesy)
var2=rss2/(n-2)
print(var2)

RSS:  556.8252631344158
2.8122488037091706


In [166]:
xmed2 = np.mean(X)
seb_2=((var2)*((1/n1)+(((xmed2)**2)/(np.sum((X-xmed2)**2)))))**0.5
print("Error estandar de beta 0: ",seb_2)

Error estandar de beta 0:  0.12558553998049232


In [168]:
seb1_2=(var2/(np.sum((X-xmed2)**2)))**0.5
print("Error estandar de beta 1: ",seb1_2)

Error estandar de beta 1:  0.0008195361847645331


In [170]:
t1_2=coef/seb1
print("Valor t de beta 1: ",t1_2)

Valor t de beta 1:  [  0.          55.84212804 230.04177286  -1.2652199 ]


In [172]:

p_ridge=2*(1-stats.t.cdf(np.abs(t1_2),n1-2))
print("p-value: ", p_ridge)

p-value:  [1.         0.         0.         0.20727945]


In [174]:
y_pred_ridge = ridge.predict(X)
r2=r2_score(salesy,y_pred_ridge)
print("R^2: ", r2)

R^2:  0.8972106381360829


## Con Penalización L1

In [177]:
lasso = Lasso(alpha=0.5)
lasso.fit(X, salesy)
interla=lasso.intercept_
print("Intercepto: ",interla) 

Intercepto:  2.980656625064583


In [179]:
coefla=lasso.coef_
print("Coeficientes: ",coefla)

Coeficientes:  [ 0.          0.04570812  0.18572931 -0.        ]


In [181]:
rss1=np.sum((salesy-lasso.predict(X))**2)
print("RSS: ",rss1)
n=len(salesy)
var1=rss1/(n-2)
print(var1)

RSS:  557.1451398714054
2.8138643427848757


In [183]:
xmed1 = np.mean(X)
seb0=((var1)*((1/n1)+(((xmed1)**2)/(np.sum((X-xmed1)**2)))))**0.5
print("Error estandar de beta 0: ",seb0)

Error estandar de beta 0:  0.1256216070626806


In [185]:
seb1_1=(var1/(np.sum((X-xmed1)**2)))**0.5
print("Error estandar de beta 1: ",seb1_1)

Error estandar de beta 1:  0.0008197715484770813


In [187]:
t1_1=coefla/seb1_1
print("Valor t de beta 1: ",t1_1)

Valor t de beta 1:  [  0.          55.75714767 226.56227811  -0.        ]


In [189]:

p_lasso=2*(1-stats.t.cdf(np.abs(t1_1),n1-2))
print("p-value: ", p_lasso)

p-value:  [1. 0. 0. 1.]


In [191]:
y_pred_lasso=lasso.predict(X)
r2_lasso=r2_score(salesy,y_pred_lasso)
print("R^2: ",r2_lasso)

R^2:  0.8971515892246072


Podemos observar que la R^2 nos dio la misma en los calculos de la regresión sin penalización y L2, con un valor de 0.89721, mientras que en la L1 nos da 0.89715. Así mismo, los valores de intercepto y los coeficientes son iguales en L2 y en la sin penalización. Sin embargo, en L1 encontramos datos distintos, ya que, en L1 puede llevar a los coeficientes a ser exactamente 0, considerando que estos coeficientes de estas variables no son significativas.